In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
from sqlalchemy import create_engine
from pysandag.database import get_connection_string

db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

### 1. Get output of simulation

##### urbansim.urbansim.urbansim_lite_output (unit change by year, grouped by jurisdiction)

In [ ]:
run_id = 2

In [ ]:
hs_change_sql = '''
    SELECT j.name, p.jurisdiction_id, sum(units_added) as hs_change, year_simulation
      FROM urbansim.urbansim.urbansim_lite_output o 
      JOIN urbansim.urbansim.parcel p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.jurisdiction_id = j.jurisdiction_id
     WHERE run_id =  %s
  GROUP BY j.name,p.jurisdiction_id, year_simulation
  ORDER BY j.name,p.jurisdiction_id, year_simulation'''
hs_change_sql = hs_change_sql % run_id

In [ ]:
units_by_jur = pd.read_sql(hs_change_sql,mssql_engine,index_col='jurisdiction_id')

### 2. Fill in "0" for units for "missing" years (for plotting) (e.g. Del Mar)

In [ ]:
idx = range(2017,2051)

In [ ]:
# Del Mar example
del_mar_before = units_by_jur.loc[4].sort_values(by='year_simulation')
del_mar_before.head()

In [ ]:
del_mar_before.plot(x='year_simulation',y='hs_change',style='.-',title='Del Mar Housing Unit Change by Year')

In [ ]:
units_by_jur.set_index(['name','year_simulation'],append=True,inplace=True)
units_by_jur = units_by_jur.unstack(['jurisdiction_id','name'])
units_by_jur = units_by_jur.reindex(idx, fill_value=0)
units_by_jur.fillna(0,inplace=True)
units_by_jur = units_by_jur.stack(['jurisdiction_id','name'])
units_by_jur.reset_index(inplace=True)
units_by_jur.set_index('jurisdiction_id',inplace=True)

In [ ]:
# Del Mar example
del_mar_after = units_by_jur.loc[4].sort_values(by='year_simulation')
del_mar_after.head()

In [ ]:
del_mar_after.plot(x='year_simulation',y='hs_change',style='.-',title='Del Mar Housing Unit Change by Year')

### 3. Get capacity

#####  <span style="color:red">!!! NOTE: capacity from urbansim.parcel does not match capacity for sched dev from sched dev table</span>

##### from urbansim.parcel (site_id IS NULL)

In [ ]:
capacity_by_jurs_sans_sched_dev_sql = '''  
    SELECT jurisdiction_id,  sum(capacity) as capacity_sans_sched_dev
      FROM urbansim.parcel
     WHERE site_id IS NULL and capacity > 0
  GROUP BY jurisdiction_id
  ORDER BY jurisdiction_id'''

In [ ]:
cap_by_jur_sans_sched =  pd.read_sql(capacity_by_jurs_sans_sched_dev_sql,mssql_engine,index_col='jurisdiction_id')

##### from urbansim.scheduled_development_do_not_use (capacity for sched dev)

In [ ]:
# Note: only count sched dev with year >= 2017
capacity_by_jurs_sched_dev_sql = '''  
    SELECT jurisdiction_id,  sum(res_units) as capacity_only_sched_dev
      FROM urbansim.urbansim.scheduled_development_do_not_use s
      JOIN urbansim.parcel p on p.parcel_id = s.parcel_id
     WHERE yr >= 2017
  GROUP BY jurisdiction_id
  ORDER BY jurisdiction_id'''

In [ ]:
cap_by_jur_sched_dev =  pd.read_sql(capacity_by_jurs_sched_dev_sql,mssql_engine,index_col='jurisdiction_id')

##### add capacity from two tables

In [ ]:
cap_by_jur = cap_by_jur_sans_sched.join(cap_by_jur_sched_dev)

In [ ]:
cap_by_jur['capacity'] = cap_by_jur['capacity_sans_sched_dev'] + cap_by_jur['capacity_only_sched_dev']

In [ ]:
cap_by_jur = cap_by_jur.drop(['capacity_sans_sched_dev', 'capacity_only_sched_dev'],axis=1)

### 4. Sum units from output of simulation over five year increments

In [ ]:
bins = range(2015,2055,5)
names = [str(x) for x in range(2020,2055,5)]
units_by_jur['increment'] = pd.cut(units_by_jur.year_simulation, bins, labels=names)

In [ ]:
units_by_jur_inc = pd.DataFrame({'hs_added': units_by_jur.
                                            groupby(["increment", "jurisdiction_id","name"]).
                                 hs_change.sum()}).reset_index()

### 5. Join output of simulation with capacity

In [ ]:
units_by_jur_inc = units_by_jur_inc.set_index('jurisdiction_id').join(cap_by_jur)

### 6. Add increment 2017 with units added equal to 0 (for plotting)

In [ ]:
start_year = units_by_jur_inc.loc[units_by_jur_inc.increment=='2020'].copy()

In [ ]:
start_year['increment'] = '2017'
start_year['hs_added'] = 0

In [ ]:
units_by_jur_inc = pd.concat([start_year,units_by_jur_inc])

### 7. Pivot so each jurisdiction is a column, each row is an increment and the values are hs unit change

In [ ]:
units_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='name', values='hs_added').\
reset_index().rename_axis(None, axis=1)
units_by_jur_pivot.fillna(0,inplace=True)
units_by_jur_pivot.set_index('increment',inplace=True)

### 8. Pivot so each jurisdiction is a column, each row is an increment and the values are capacity

In [ ]:
cap_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='name', values='capacity').\
reset_index().rename_axis(None, axis=1)
cap_by_jur_pivot.fillna(0,inplace=True)
cap_by_jur_pivot.set_index('increment',inplace=True)

### 9. Calculate cumulative sum of hs unit change by jurisdiction

In [ ]:
units_by_jur_pivot = units_by_jur_pivot.cumsum()

### 10. Plot results - line plots

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=10, ncols=2)
for j, jur in enumerate(units_by_jur.name.unique().tolist()):
    df_units_added = units_by_jur_pivot[[jur]]
    df_capacity = cap_by_jur_pivot[[jur]]
    # join dataframe of units added with dataframe of capacity for each jurisdiction
    df_plot = df_units_added.join(df_capacity, lsuffix='_hs_change', rsuffix='_capacity')
    hs_column = jur + '_hs_change'
    cap_column = jur + '_capacity'
    cap_plot_column = jur + '_remaining_cap'
    # calculate remaining capacity by subtracting
    # units added from starting capacity
    df_plot[cap_plot_column] = df_plot[cap_column] - df_plot[hs_column]
    del  df_plot[cap_column]
    df_plot.plot(style='.-',ax=axes.flat[j],figsize=(10,32))
    fig.savefig('units_and_capacity_lineplot.png', format='png', dpi=300)

### 10. Plot results - bar plots

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=2)
for j, jur in enumerate(units_by_jur.name.unique().tolist()):
    df_units_added = units_by_jur_pivot[[jur]]
    df_capacity = cap_by_jur_pivot[[jur]]
    # join dataframe of units added with dataframe of capacity for each jurisdiction
    df_plot = df_units_added.join(df_capacity, lsuffix='_hs_change', rsuffix='_capacity')
    hs_column = jur + '_hs_change'
    cap_column = jur + '_capacity'
    cap_plot_column = jur + '_remaining_cap'
    # calculate remaining capacity by subtracting
    # units added from starting capacity
    df_plot[cap_plot_column] = df_plot[cap_column] - df_plot[hs_column]
    del  df_plot[cap_column]
    df_plot.plot(style='.-',ax=axes.flat[j],figsize=(10,32),kind='bar')
    fig.savefig('units_and_capacity_barplot.png', format='png', dpi=300)

### 11. Double check results for one jurisdiction: Encinitas

Plot of Encinitas housing unit change and remaining capacity

In [ ]:
jur = 'Encinitas'
df_units_added = units_by_jur_pivot[[jur]]
df_capacity = cap_by_jur_pivot[[jur]]
df_plot = df_units_added.join(df_capacity, lsuffix='_hs_change', rsuffix='_capacity')
hs_column = jur + '_hs_change'
cap_column = jur + '_capacity'
cap_plot_column = jur + '_remaining_cap'
df_plot[cap_plot_column] = df_plot[cap_column] - df_plot[hs_column]
del  df_plot[cap_column]
df_plot.plot(style='.-')

Capacity and housing unit change shown on plot

In [ ]:
print("Starting capacity: {}\nHousing unit change: {}\nRemaining:   {}".\
      format(int(df_plot.loc['2017'].Encinitas_remaining_cap),\
       int(df_plot.loc['2050'].Encinitas_hs_change),\
             int(df_plot.loc['2017'].Encinitas_remaining_cap)-int(df_plot.loc['2050'].Encinitas_hs_change)))
print("Remaining capacity Encinitas 2050: {}".format(int(df_plot.loc['2050'].Encinitas_remaining_cap)))

Capacity on confluence page Jurisdiction Feedback

In [ ]:
print("Starting capacity on jur feedback confluence page: 2460")
print("Difference between confluence page and plot: {}".\
      format(int(df_plot.loc['2017'].Encinitas_remaining_cap)-2460))

Capacity and forecast from database

In [ ]:
encinitas_capacity_sql =  '''
     SELECT sum(capacity)
       FROM urbansim.parcel
      WHERE jurisdiction_id = 6 and capacity > 0'''
cap_encinitas_df =  pd.read_sql(encinitas_capacity_sql,mssql_engine)
encinitas_capacity_sql_no_sched_dev =  '''
     SELECT sum(capacity)
       FROM urbansim.parcel
      WHERE jurisdiction_id = 6 and site_id IS NULL and capacity > 0'''
cap_encinitas_df_no_sched_dev =  pd.read_sql(encinitas_capacity_sql_no_sched_dev,mssql_engine)
encinitas_capacity_sql_just_sched_dev =  '''
     SELECT sum(res_units)
       FROM urbansim.urbansim.scheduled_development_do_not_use s
       JOIN urbansim.parcel p on p.parcel_id = s.parcel_id
      WHERE jurisdiction_id = 6 and yr >= 2017'''
simulation_sched_dev =  '''  
    SELECT  sum(units_added)
       FROM urbansim.urbansim.urbansim_lite_output o
       JOIN urbansim.parcel p on p.parcel_id = o.parcel_id
      WHERE jurisdiction_id = 6 and source='sched_dev' and run_id =  %s'''
simulation_sched_dev_sql = simulation_sched_dev % run_id
sim_sched_dev_df =  pd.read_sql(simulation_sched_dev_sql,mssql_engine)
simulation_NOT_sched_dev =  '''  
    SELECT  sum(units_added)
       FROM urbansim.urbansim.urbansim_lite_output o
       JOIN urbansim.parcel p on p.parcel_id = o.parcel_id
      WHERE jurisdiction_id = 6 and source != 'sched_dev' and run_id =  %s'''
simulation_NOT_sched_dev_sql = simulation_NOT_sched_dev % run_id
sim_NOT_sched_dev_df =  pd.read_sql(simulation_NOT_sched_dev_sql,mssql_engine)

In [ ]:
print("\nTotal capacity from urbansim.parcel: {}".\
      format(int(cap_encinitas_df.values)))
print("Total capacity from urbansim.parcel and sched dev tbale: {}\n".\
      format(int(cap_encinitas_df_no_sched_dev.values) +\
             int(cap_encinitas_df_just_sched_dev.values)))
print("Capacity sched dev: {}".\
      format(int(cap_encinitas_df_just_sched_dev.values)))
print("Forecast sched dev: {}\n".\
      format(int(sim_sched_dev_df.values)))

print("Capacity no sched dev: {}".\
      format(int(cap_encinitas_df_no_sched_dev.values)))
print("Forecast no sched dev: {}\n".\
      format(int(sim_NOT_sched_dev_df.values)))

print("Remaining: {}".\
      format(int(cap_encinitas_df_no_sched_dev.values)-\
                 int(sim_NOT_sched_dev_df.values)))
print("Remaining shown on plot: {}\n".format(int(df_plot.loc['2050'].Encinitas_remaining_cap)))
print("Forecast total: {}".\
      format(int(sim_sched_dev_df.values) +  int(sim_NOT_sched_dev_df.values)))
print("Forecast housing unit change on plot: {}".\
      format(int(df_plot.loc['2050'].Encinitas_hs_change)))


In [ ]:
remaining_capacity_sql = ''' 

        WITH parcel_out AS 
            (SELECT  o.parcel_id, sum(units_added) as units_added
                FROM urbansim.urbansim.urbansim_lite_output o 
                JOIN urbansim.parcel p on p.parcel_id = o.parcel_id
                WHERE jurisdiction_id = 6 and source != 'sched_dev' and  run_id =  %s
                GROUP BY o.parcel_id)
        SELECT sum(p.capacity - COALESCE(o.units_added,0))
        FROM urbansim.parcel p
        FULL OUTER JOIN parcel_out o
        ON o.parcel_id = p.parcel_id
        WHERE jurisdiction_id = 6 and site_id IS NULL and p.capacity  > 0 '''
remaining_capacity_sql = remaining_capacity_sql % run_id
remaining_capacity_df =  pd.read_sql(remaining_capacity_sql,mssql_engine)
remaining_capacity_df =  pd.read_sql(remaining_capacity_sql,mssql_engine)

In [ ]:
print("Remaining capacity from db: {}".\
      format(int(remaining_capacity_df.values)))